<h2>Slitmask Spectroscopic Reduction: ARC 3.5m KOSMOS

Notebook goes through basic ideas of reducing KOSMOS slitmask spectra. Does not discuss basic reduction (see KOSMOS notebook) but concentrates on multi-slit issues: finding the slitlets, doing wavelength calibration for each slitlet (trickiest part), 2D extraction of slitlits, including wavelength correction by skylines and undistorting along slit to central wavelength, and simple 1D extraction.

In [1]:
from pyvista import imred, tv, stars, slitmask, image, spectra
import pdb
import copy
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.table import vstack

In [2]:
# you may need/want to use qt or osx in the next line
%matplotlib tk    
t=tv.TV()

In [3]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/raw/apo/UT221221_cleaned',verbose=False)                                                         
flat = red.reduce(71)
arc = red.reduce(70)
arcs= red.sum([70,67])

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Find slit edges from flat, and fit polynomials to locations

In [4]:
trace=spectra.Trace(transpose=True)
t.tvclear()
bottom,top = trace.findslits(flat,display=t,thresh=0.5,sn=True)

Plot shape of slit edges relative to center position, just for information

In [13]:
x=np.arange(4096)
plt.figure()
for l in bottom :
    plt.plot(x,l(x)-l(2048))
    plt.xlabel('x')
    plt.ylabel('y-y(2048)')

Read in the slitmask file, which we'll use to get object names and slit locations to help with wavelength solution.

If the number of slitlets found doesn't match the number of targets, you'll need to go back and adjust the threshold to find the correct slitlets, or else modify the targets table below so that they match.

In [5]:
kmsfile='eFEDS197_pointing2.1.kms'
targets = slitmask.read_kms(kmsfile,sort='YMM')
if len(targets) != len(bottom) : 
    print('ERROR, number of identified slits does not match number of targets')
targets

ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str5,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG114,NN,STRAIGHT,4.0,4.0,0.0,83654.162,30638.76,0.683,0.683,-6.272,-33.306
TARG111,NN,STRAIGHT,4.0,4.0,0.0,83645.269,30628.86,0.683,0.683,16.462,-31.616
TARG109,Obj9,STRAIGHT,2.0,10.0,0.0,83702.29,30538.62,0.341,1.707,-27.05,-23.042
TARG101,Obj1,STRAIGHT,2.0,10.0,0.0,83649.106,30504.92,0.341,1.707,6.652,-17.29
TARG103,Obj3,STRAIGHT,2.0,10.0,0.0,83654.799,30436.19,0.341,1.707,-7.901,-12.387
TARG102,Obj2,STRAIGHT,2.0,10.0,0.0,83638.93,30417.69,0.341,1.707,32.666,-9.23
TARG104,Obj4,STRAIGHT,2.0,10.0,0.0,83655.258,30349.0,0.341,1.707,-9.075,-4.333
TARG105,Obj5,STRAIGHT,2.0,10.0,0.0,83648.518,30323.98,0.341,1.707,8.155,-0.063
TARG106,Obj6,STRAIGHT,2.0,10.0,0.0,83644.95,30258.99,0.341,1.707,17.277,4.202


In [6]:
# if you want subset of slitlets, e.g. ignoring alignments stars, select them here
gd=np.where(targets['NAME']!='NN')[0]     # or, e.g., gd=[2,3,4,7,9]
gdtrace=copy.deepcopy(trace)
gdtrace.model = [trace.model[i] for i in gd]
gdtrace.rows = [trace.rows[i] for i in gd]
trace=copy.deepcopy(gdtrace)
targets=targets[gd]
print(targets)
vars(trace)

   ID    NAME  SHAPE   WID LEN  ROT ...  DELTA   WIDMM LENMM  XMM     YMM  
------- ----- -------- --- ---- --- ... -------- ----- ----- ------ -------
TARG109  Obj9 STRAIGHT 2.0 10.0 0.0 ... 30538.62 0.341 1.707 -27.05 -23.042
TARG101  Obj1 STRAIGHT 2.0 10.0 0.0 ... 30504.92 0.341 1.707  6.652  -17.29
TARG103  Obj3 STRAIGHT 2.0 10.0 0.0 ... 30436.19 0.341 1.707 -7.901 -12.387
TARG102  Obj2 STRAIGHT 2.0 10.0 0.0 ... 30417.69 0.341 1.707 32.666   -9.23
TARG104  Obj4 STRAIGHT 2.0 10.0 0.0 ...  30349.0 0.341 1.707 -9.075  -4.333
TARG105  Obj5 STRAIGHT 2.0 10.0 0.0 ... 30323.98 0.341 1.707  8.155  -0.063
TARG106  Obj6 STRAIGHT 2.0 10.0 0.0 ... 30258.99 0.341 1.707 17.277   4.202
TARG110 Obj10 STRAIGHT 2.0 10.0 0.0 ... 30145.73 0.341 1.707  7.319  16.706
TARG107  Obj7 STRAIGHT 2.0 10.0 0.0 ... 30031.64 0.341 1.707 -0.036   29.35
TARG108  Obj8 STRAIGHT 2.0 10.0 0.0 ...  30001.4 0.341 1.707  0.649  34.511


{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'model': [Polynomial([455.39492819,   6.86250423, -10.5109358 ], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([584.86096608,   6.98188667,  -7.20510802], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([695.30089454,   7.03335661,  -4.50728817], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([769.89113187,   7.06478212,  -2.53785573], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([ 8.78061651e+02,  7.09096738e+00, -1.05166117e-01], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([975.73794736,   7.10712024,   2.25961684], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1072.80140623,    7.06138006,    4.58015777], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1356.17765073,    7.06397733,   11.243

Using the derived traces, extract the slitlets for arcs

In [7]:
arcec=trace.extract2d(arcs,display=t)

extracting: 
 455-494
 585-624
 695-734
 770-809
 878-917
 976-1015
 1073-1112
 1356-1395
 1642-1680
 1758-1796
  See extraction window(s). Hit space bar to continue....


Add XMM and YMM for each slit to headers of each extracted image. They need to match so you get the right values!

In [8]:
for arc,target in zip(arcec,targets) : 
    arc.header['XMM'] = target['XMM']
    arc.header['YMM'] = target['YMM']

Now loop through each extracted arc to do wavelength calibration. This requires a little effort because the change in the location of the slit relative to the default saved wavelength calibration is significant enough that it can be a challenge to automatically find the lines, since the change in spectrum is more than a simple shift (and, in fact, more than a shift + dispersion change). 

However, a simple shift is usually enough to identify some of the lines, and these can be used to bootstrap the wavelength solution; the initial identification is easier if you use an estimate of the shift from the mask design XMM.

You can use identify() to do the iteration. On the first pass, only central lines may be correctly identified. Use 'l' and 'r' to remove lines to the left and right of the identified lines. Then use 'i' to iterate, i.e., allow it to re-identify lines (i just returns True to allow you to iterate). When happy with solution, use ' ' to move onto the final 2D wavelength calibration.

You can really help this process if you supply an initial wavelength calibration (a pyvista WaveCal object) that was done using the same lamp(s) as your arc exposures (here, using KOSMOS_red_waves.fits'), and using a master line list that corresponds to these lamp(s) (here, using ne.dat). If you choose another WaveCal to start from, you may need to get a correct approximate relation for the shift from the reference spectrum as a function of XMM. For KOSMOS, seems like -22.5(XMM) gives a rough pixel shift from a center slit location, -22.5(XMM-24.44) for a low slit location.

The next cell just shows how the simple shift fails, even using the XMM for each slit to shift: all of the arc lines don't match up with just a translation.

In [16]:
plt.figure()
wav2=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
plt.plot(wav2.spectrum[0])
for i, arc in enumerate(arcec[0:1]) :
    shift=(arc.header['XMM']*-22.5)
    plt.plot(arc.data[19][int(shift):]*30)
    print(shift)
    plt.draw()

  rms:    0.177 Angstroms (50 lines)
608.625


The wavelength calibration for each slitlet will be saved. Since this is probably the most time-consuming part, you could use saved one if you only want to redo a couple of them (sometimes, you hit the wrong key and want to do one over!)

In [17]:
clobber=False         # set to False if you want to use any saved ones
for i,(arc,targ) in enumerate(zip(arcec,targets)) :
    wavname = 'wav_{:s}.fits'.format(targ['NAME'])
    if clobber or not os.path.exists(wavname) :
        wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
        wav.fit(degree=3)
        nrow=arc.shape[0]
    
        # get initial guess at shift from reference using XMM (KOSMOD red low!)
        shift=int(arc.header['XMM']*-22.5) # +550  #-wav.pix0)
        lags=np.arange(shift-20,shift+20)

        iter = True
        while iter :
            iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,lags=lags,thresh=10,file='henear.dat')
            lags=np.arange(-50,50)
            plt.close()

        bd= np.where(wav.weights<0.5)[0]
        print(wav.waves[bd])
        # Do the 2D wavelength solution, sampling 10 locations across slitlet
        wav.degree=5
        wav.identify(arc,plot=True,nskip=nrow//10,thresh=10)
        plt.close()
        wav.write(wavname)
        wav.add_wave(arc)
        t.tv(wav.correct(arc,arc.wave[nrow//2]))

Now set up routine to reduct/extract science frames

In [12]:
def multi_extract2d(red,trace,targets,image,bias=None,dark=None,flat=None,display=None,crbox='lacosmic',crsig=10,
                    rad=5) :

    if display is not None : 
        display.clear()
        plot = True
    else:
        plot = False
        
    # basic image read and reduction
    imcr=red.reduce(image,bias=bias,dark=dark,flat=flat,crbox=crbox,display=display,crsig=crsig)

    # 2D extraction
    out = trace.extract2d(imcr,display=display)

    # loop over each desired slitlet, get wavelength shift from skylines, undistort to central wavelengths
    diff=[]
    for i,(o,targ) in enumerate(zip(out,targets)) :
        wav=spectra.WaveCal('./wav_{:s}.fits'.format(targ['NAME']))
        orig=wav.model.c0_0
        wav.add_wave(o)

        # set rows to use for skyline
        nrows = o.shape[0]
        rows = [x for x in range(0,nrows) if np.abs(x-nrows//2)>rad]
        wav.skyline(o,thresh=10,rows=rows,plot=plot)
        
        if plot : plt.close()
        wav.add_wave(o)
        if display is not None : display.tv(o)
        out[i]=wav.correct(o,o.wave[nrows//2])
        if display is not None : display.tv(out[i])
        name = out[i].header["FILE"].split(".")[0] 
        out[i].write('{:s}_{:s}_2d.fits'.format(name,targ['NAME']))
        diff.append(wav.model.c0_0-orig)

    print('shifts: ',diff)
    return out


In [13]:
out=multi_extract2d(red,trace,targets,54,display=None)

  starting CR rejection, may take some time ....
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
extracting: 
 455-494
 585-624
 695-734
 770-809
 878-917
 976-1015
 1073-1112
 1356-1395
 1642-1680
 1758-1796
  rms:    0.319
rejecting 6 points from 243 total: 
  rms:    0.319
rejecting 6 points from 243 total: 
  rms:    0.341
rejecting 0 points from 187 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.221
rejecting 4 points from 555 total: 
  rms:    0.221
rejecting 4 points from 555 total: 


/home/holtz/git/pyvista/python/pyvista/spectra.py:708: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  rms:    0.850
rejecting 3 points from 324 total: 
  rms:    0.629
rejecting 4 points from 324 total: 
  rms:    0.613
rejecting 4 points from 324 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.358
rejecting 11 points from 309 total: 
  rms:    0.358
rejecting 11 points from 309 total: 
  rms:    0.617
rejecting 4 points from 266 total: 
  rms:    0.451
rejecting 4 points from 266 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.286
rejecting 7 points from 249 total: 
  rms:    0.286
rejecting 7 points from 249 total: 
  rms:   10.756
rejecting 1 points from 293 total: 
  rms:    1.133
rejecting 1 points from 293 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.349
rejecting 13 points from 313 total: 
  rms:    0.349
rejecting 13 points from 313 total: 
  rms:  320.534
rejecting 1 points from 251 total: 
  rms:    1.012
rejecting 1 points from 251 total: 

appending uncertainty
appending bitmask
appendi

As desired, move on to 1D extraction

In [16]:
def multi_extract1d(spec2d) :
    def model(x) :
        return x*0.

    fig=plt.figure()
    spec1d=[]
    for i in range(len(spec2d)) :
        trace1 = spectra.Trace(transpose=False)
        trace1.rows = [0,spec2d[i].data.shape[0]]
        trace1.index = [0]
        peak,ind = trace1.findpeak(spec2d[i],thresh=10,sort=True)
        if len(peak) > 0:
            def model(x) :
                return x*0. + peak[0]
            trace1.model = [model]
            spec=trace1.extract(spec2d[i],rad=4,back=[[-10,-5],[5,10]],display=None)
            plt.figure(fig)
            spec.wave = out[i].wave[peak]
            print(spec.wave[0].shape,spec.data[0].shape)
        
            plt.plot(spec.wave[0],spec.data[0])
            plt.plot(spec.wave[0],spec.sky[0])
            spec1d.append(spec)
        else :
            print('no peak found for slit: ',i)
    return spec1d

In [17]:
spec1d = multi_extract1d(out)

looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [8, 34]
aperture/fiber:  [0, 1]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [21, 33]
aperture/fiber:  [0, 1]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [18]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
